In [49]:
import pandas as pd

addr = pd.read_csv("../volgait2024-semifinal-addresses.csv", sep=';')
df = pd.read_csv("../volgait2024-semifinal-task.csv", sep=';')
streets = list(addr['house_full_address'])


street_to_uuid = dict(zip(addr['house_full_address'], addr['house_uuid']))

In [55]:
from gliner import GLiNER

model = GLiNER.from_pretrained("models/checkpoint-1500")
# base_model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

for text in df['comment'].sample():
    labels = ["Полный адрес"]

    entities = model.predict_entities(text, labels, threshold=0.1)

    print()
    print(text)
    print()
    for entity in entities:
        print(entity["text"], "=>", entity["label"])
        
        sublabels = ['street name (without house numbers)']
        subentities = base_model.predict_entities(entity['text'], sublabels, threshold=0.3)

        for subentitiy in subentities:
            print('\t', subentitiy['text'], '->', subentitiy['label'])

        # print(find_street_names_and_numbers(entity['text']))



    
    print('\n\n')

    break

config.json not found in C:\Users\admin\Desktop\semifinal\gliner_train\models\checkpoint-1500
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Ульяновск г, Луначарского ул, д. 3, 11, 5, 9, 7 Ульяновск г, Гая пр-кт, д. 21 Ульяновск г, Соловьева ул, д. 1

Гая пр-кт, д. 21 => Полный адрес
	 Гая пр-кт -> street name (without house numbers)
Соловьева ул, д. 1 => Полный адрес
	 Соловьева ул -> street name (without house numbers)





In [27]:
import re

def is_street_name(part):
    """
    Determine if the given part is a street name by checking if it contains more letters than digits
    and has a minimum length of 3 characters.

    Parameters:
    part (str): The part to check.

    Returns:
    bool: True if the part is a street name, False otherwise.
    """
    letters = sum(c.isalpha() for c in part)
    digits = sum(c.isdigit() for c in part)
    return letters > digits and len(part) >= 3

def find_street_names_and_numbers(text):
    """
    Find street names and house numbers in the given text by splitting the text and analyzing each part.

    Parameters:
    text (str): The text to search for street names and house numbers.

    Returns:
    tuple: A tuple containing the street name and a list of house numbers or 'all'.
    """
    # Split the text into parts using a regular expression to handle multiple delimiters
    parts = re.split(r'[ ,]+', text)
    street_name = None
    house_numbers = []

    for part in parts:
        if is_street_name(part):
            if street_name:
                if not house_numbers:
                    house_numbers = 'all'
                return (street_name, house_numbers)
            street_name = part
        elif part.isdigit() or re.match(r'\d+[а-яА-Я]?', part):
            house_numbers.append(part)

    if not house_numbers:
        house_numbers = 'all'

    return (street_name, house_numbers)

# Example usage:
text1 = "Тухачевского/Федерации утечка из-под земли д=150. без ХВС пер Федерации с 34 по 40.ул Северная с 3 по11. ул Федерации  с 141 по193"
text2 = "Ленина"

result1 = find_street_names_and_numbers(text1)
result2 = find_street_names_and_numbers(text2)

print(result1)  # Output: ('Тухачевского/Федерации', 'all')
print(result2)  # Output: ('Ленина', 'all')
print(find_street_names_and_numbers("ул Северная с 3 по11"))

('Тухачевского/Федерации', 'all')
('Ленина', 'all')
('Северная', ['3'])
